# Week VI: Sequence Analysis II - Sequence Alignments & Phylogenetics

## Sequence Alignment

One of the fundamental challenges in bioinformatics involves assessing the similarity between biological sequences. This task has various applications, such as deducing the biological function of a novel protein sequence, determining the source organism of a given DNA sequence, and engaging in molecular phylogenetics to formulate hypotheses about the relationships between organisms. At first glance, this may appear to be a straightforward problem, seemingly not warranting decades of research or becoming the focus of one of the most cited papers in modern biology.

### Alignment Tools

Numerous algorithms exist for aligning sequences, encompassing both _pairwise alignments_ and _multiple sequence alignments_. These computations are generally time-intensive, making it impractical to implement such algorithms directly in Python. For pairwise alignments, Biopython provides the `PairwiseAligner` module. Additionally, Biopython allows you to execute command line tools seamlessly. Typically, the process involves:

1. Preparing an input file containing your unaligned sequences, often formatted as a FASTA file, which you can create using `Bio.SeqIO`.
2. Invoking the command line tool to process the input file, usually through one of Biopython's command line wrappers.
3. Reading the output from the tool, which comprises your aligned sequences, often accomplished using `Bio.AlignIO`.

All the command line wrappers discussed here follow a similar structure. You create a command line object, specifying options (e.g., input and output filenames), and then execute this command line through a Python operating system call (e.g., utilizing the `subprocess` module).

The majority of these wrappers are defined within the `Bio.Align.Applications` module:

In [ ]:
import Bio.Align.Applications
dir(Bio.Align.Applications)[0:9]

### What is a sequence similarity?

Imagine you have three protein sequences - call them ``seq1``, ``seq2`` and ``seq3`` - and you want to know whether ``seq3`` is more similar to ``seq1`` or ``seq2``. At first glance, it may appear feasible to determine the dissimilarity by simply counting the positions where the sequences differ by calculating the [Hamming distance](http://en.wikipedia.org/wiki/Hamming_distance) between them. Below is an illustration of how this could be done using an implementation of the Hamming distance from the SciPy Python library.

In [ ]:
!pip install scipy
!pip install scikit-bio

In [ ]:
from scipy.spatial.distance import hamming
import skbio

# NCBI Reference Sequence: NP_000508.1 (human hemoglobin subunit A)
seq1 = skbio.Protein("MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR")

# NCBI Reference Sequence: NP_001004376.1 (chicken hemoglobin subunit A)
seq2 = skbio.Protein("MVLSAADKNNVKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHFDLSHGSAQIKGHGKKVVAALIEAANHIDDIAGTLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVHASLDKFLCAVGTVLTAKYR")

# GenBank: QFF91579.1 (sei whale hemoglobin subunit A)
seq3 = skbio.Protein("MVLFPADKSNVKATWAKIGNHGAEYGAEALERMFMNFPSTKTYFPHFDLGHDSAQVKGHGKKVADALTKAAGHMDNLLDALSDLSDLHAHKLRVDPVNFKLLSHCLLVTLALHLPAEFTPSVHASLDKFLASVSTVLTSKYR")

print(seq1)
print(seq2)
print(seq3)

Here we stored 3 protein sequences as variables. Now we're going to compute the hamming distance between them using the function `hamming()` to assess whether our query sequence (seq3) is closer to reference sequence 1 (seq1) or reference sequence 2 (seq2) based on the hamming distance:

In [ ]:
print(hamming(seq1, seq3))
print(hamming(seq2, seq3))

In this instance, the distance between 'seq3' and 'seq1' is smaller than that between 'seq3' and 'seq2,' indicating that 'seq3' is more similar to 'seq1' than to 'seq2.' The comment lines of each sequence provide information about their source organisms. The Hamming distances observed align with our knowledge of the evolutionary distances between these organisms. For instance, the whale hemoglobin exhibits a smaller Hamming distance (indicating greater similarity) to the human hemoglobin sequence than to the chicken hemoglobin sequences. Since whales and humans are both mammals, while chickens are birds, whales and humans are more closely related to each other than either is to birds. If we had postulated that whales were more closely related to humans than to chickens, this result would have supported that hypothesis. However, it's important to note that relating sequence similarity to evolutionary history is not always straightforward.

### What is a sequence alignment? 

Let's take a moment to reflect on sequence evolution and understand what a biological sequence alignment entails. Throughout biological evolution, DNA sequences undergo changes, often resulting from random errors in replication, commonly known as **mutations**. Some notable types of mutation events include:

* **Substitutions:** This occurs when one DNA base is replaced with another.
* **Insertions:** Involves the insertion of one or more contiguous DNA bases into a sequence.
* **Deletions:** Encompasses the removal of one or more contiguous DNA bases from a sequence.

While other types of mutation events exist, our current focus will be on these mentioned types.

**Note:** In the protein coding regions of genomes, substitutions may or may not lead to a modification in the resulting protein. These mutations are categorized as non-synonymous or synonymous mutations, depending on whether they affect the protein. On the other hand, insertions or deletions will alter the resulting protein, and the magnitude of the insertion or deletion (i.e., the number of bases inserted or deleted) determines the extent of the impact the mutation has on the resulting protein sequence.

#### An Initial Algorithm for Aligning a Pair of Sequences

We will establish two sequences, "seq1" and "seq2," and subsequently outline an approach for aligning them. In this section, our focus will be on DNA sequences, while future sections will delve into aligning protein sequences. It's worth noting that the algorithms employed are generally applicable to both DNA/RNA and protein sequences. Throughout this book and your own endeavors, you may encounter scenarios where alignment is performed on DNA or RNA sequences, as well as on protein sequences.

In [12]:
from skbio import DNA
seq1 = DNA("ACCGGTGGAACCGGTAACACCCAC")
seq2 = DNA("ACCGGTAACCGGTTAACACCCAC")

##### Step 1: Create a blank matrix where the rows and columns represent the positions in the sequences. 

We'll create this matrix and initialize it with all zeros as follows:

In [14]:
import numpy as np

num_rows = len(seq2)
num_cols = len(seq1)
data = np.zeros(shape=(num_rows, num_cols), dtype=int)

We will use a function called `show_F` to display matrices.

In [15]:
import tabulate

def show_F(h_sequence, v_sequence, data, hide_zeros=False, nonzero_val=None):
    rows = []
    col_headers = [c.decode('UTF-8') for c in h_sequence.values]
    row_headers = [c.decode('UTF-8') for c in v_sequence.values]
    pad_headers = data.shape == (len(row_headers) + 1, len(col_headers) + 1)
    if pad_headers:
        row_headers = [" "] + row_headers
        col_headers = [" "] + col_headers
    for h, d in zip(row_headers, data):
        current_row = [h]
        for e in d:
            if e == 0:
                if hide_zeros:
                    current_row.append('')
                else:
                    current_row.append(0)
            else:
                if nonzero_val is not None:
                    current_row.append(nonzero_val)
                else:
                    current_row.append(e)
        rows.append(current_row)
    return tabulate.tabulate(rows, headers=col_headers, tablefmt='html')

In [ ]:
show_F(seq1, seq2, data)

##### Step 2: Add values to the cells in the matrix. 

Next we'll add initial values to the cells so that if the characters at the corresponding row and column are the same, the value of the cell is changed from zero to one. We can then review the resulting matrix. For clarity, we'll have ``show_F`` hide the zero values.

In [ ]:
for row_number, row_character in enumerate(seq2):
    for col_number, col_character in enumerate(seq1):
        if row_character == col_character:
            data[row_number, col_number] = 1

show_F(seq1, seq2, data, hide_zeros=True)

##### Step 3: Identify the longest diagonals. 

Next we'll identify the longest stretches of non-zero characters, which we'll refer to here as the *diagonals*. Diagonals indicate segments of the two sequences that are identical and uninterrupted by mismatched characters (substitution events) or indel events.

We can identify the longest diagonals as follows:

In [ ]:
# create a copy of our data matrix to work with, so we
# leave the original untouched.
summed_data = data.copy()
# iterate over the cells in our data matrix, starting in
# the second row and second column
for i in range(1, summed_data.shape[0]):
    for j in range(1, summed_data.shape[1]):
        # if the value in the current cell is greater than zero
        # (i.e., the characters at the corresponding pair of
        # sequence positions are the same), add the value from the
        # cell that is diagonally up and to the left.
        if summed_data[i, j] > 0:
            summed_data[i, j] += summed_data[i-1][j-1]

show_F(seq1, seq2, summed_data, hide_zeros=True)

Next, we'll identify the length of the longest diagonal.

In [ ]:
longest_diagonal_length = summed_data.max()
print("The longest diagonal is %d characters long." % longest_diagonal_length)

##### Step 4: Transcribe some of the possible alignments that arise from this process. 

We will summarize the algorithm briefly for now, and we'll revisit it in more detail later. In essence, the goal is to begin with the longest diagonal and trace it backward, transcribing the alignment by noting the characters from each of the two sequences at every corresponding row and column along the diagonal. When encountering a break in the diagonal, the next longest diagonal is identified, starting in a cell that is either up and/or to the left of the previous diagonal's endpoint. Moving straight upwards involves inserting a gap in the sequence on the horizontal axis of the matrix for every such cell, while moving straight leftwards results in a gap on the vertical axis.

Additionally, it's common to calculate a score for an alignment to assess its quality. **As a simple scoring system, we might add one for every match and subtract one for every mismatch.** If this step appears confusing at the moment, don't worry. We'll be back to this in a lot more detail soon.

Here are two possible alignments:

Alignment 1 (score: 19)
```
ACCGGTGGAACCGG-TAACACCCAC
ACCGGT--AACCGGTTAACACCCAC
```

Alignment 2 (score: 8)
```
ACCGGTGGAACCGGTAACACCCAC
ACCGGT--------TAACACCCAC
```

### Pairwise sequence alignment

Pairwise sequence alignment involves aligning two sequences to optimize their similarity score. The `Bio.Align` module includes the `PairwiseAligner` class, designed for global and local alignments. It employs algorithms such as `Needleman-Wunsch`, `Smith-Waterman`, `Gotoh (three-state)`, and `Waterman-Smith-Beyer`, offering various options to customize alignment parameters.

#### Basic usage

To generate pairwise alignments, first create a `PairwiseAligner` object:

In [20]:
from Bio import Align
aligner = Align.PairwiseAligner()

The `PairwiseAligner` object stores the alignment parameters to be used for the pairwise alignments.

These attributes can be set in the constructor of the object or after the object is made.

In [37]:
aligner = Align.PairwiseAligner(match_score=1.0)

Use the `aligner.score` method to calculate the alignment score between two sequences:

In [ ]:
target = "AGAACTC"
query = "GAACT"
score = aligner.score(target, query)
score

To see the actual alignments, use the `aligner.align` method and iterate over the Alignment objects returned:

In [ ]:
alignments = aligner.align(target, query)
for alignment in alignments:
    print(alignment)

By default, a global pairwise alignment is performed, which finds the optimal alignment over the whole length of target and query. Instead, a local alignment will find the subsequence of target and query with the highest alignment score. Local alignments can be generated by setting `aligner.mode` to **"local"**:

In [ ]:
aligner.mode = "local"
target = "AGAACTC"
query = "GAACT"
score = aligner.score(target, query)
score

In [ ]:
alignments = aligner.align(target, query)
for alignment in alignments:
    print(alignment)

#### The pairwise aligner object

The `PairwiseAligner` object stores all alignment parameters to be used for the pairwise alignments. To see an overview of the values for all parameters, use:

In [ ]:
print(aligner)

Depending on the gap scoring parameters and mode, `PairwiseAligner` object automatically chooses the appropriate algorithm to use for pairwise sequence alignment. To verify the selected algorithm, use the command below. This attribute is read-only.

In [ ]:
aligner.algorithm

#### Substitution scores

Substitution scores define the value to be added to the total score when two letters (nucleotides or amino acids) are aligned to each other. The substitution scores to be used by the `PairwiseAligner` can be specified in two ways:

By specifying a match score for identical letters, and a mismatch scores for mismatched letters. Nucleotide sequence alignments are typically based on match and mismatch scores. For example, by default _BLAST_ uses a match score of +1 and a mismatch score of −2 for nucleotide alignments by megablast, with a gap penalty of 2.5. Match and mismatch scores can be specified by setting the match and mismatch attributes of the `PairwiseAligner` object:

In [ ]:
from Bio import Align
aligner = Align.PairwiseAligner()
aligner.match_score

In [ ]:
aligner.mismatch_score

In [ ]:
score = aligner.score("ACGT", "ACAT")
print(score)

In [ ]:
aligner.match_score = 1.0
aligner.mismatch_score = -2.0
aligner.gap_score = -2.5
score = aligner.score("ACGT", "ACAT")
print(score)

Alternatively, you can use the substitution_matrix attribute of the `PairwiseAligner` object to specify a substitution matrix. This allows you to apply different scores for different pairs of matched and mismatched letters. This is typically used for amino acid sequence alignments. For example, by default _BLAST_ uses the _BLOSUM62_ substitution matrix for protein alignments by `blastp`. This substitution matrix is available from Biopython:

In [ ]:
from Bio.Align import substitution_matrices
substitution_matrices.load()

In [ ]:
matrix = substitution_matrices.load("BLOSUM62")
print(matrix)

In [ ]:
from Bio.Align import substitution_matrices
matrix = substitution_matrices.load("BLOSUM62")
aligner.substitution_matrix = matrix
score = aligner.score("ACDQ", "ACDQ")
score

In [ ]:
score = aligner.score("ACDQ", "ACNQ")
score

#### Examples

Suppose you want to do a global pairwise alignment between two sequences, prepared in FASTA format separately in _alpha.fa_ and _beta.fa_:

>HBA_HUMAN
MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHG
KKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTP
AVHASLDKFLASVSTVLTSKYR

>HBB_HUMAN
MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPK
VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFG
KEFTPPVQAAYQKVVAGVANALAHKYH

In [ ]:
from Bio import Align
from Bio import SeqIO
seq1 = SeqIO.read("alpha.faa", "fasta")
seq2 = SeqIO.read("beta.faa", "fasta")
aligner = Align.PairwiseAligner()
alignments = aligner.align(seq1.seq, seq2.seq)
alignment = alignments[0] # The alignment with hishest score is stored as first
print("Alignment Score:", alignment.score)
print(alignment)

Better alignments are usually obtained by penalizing gaps: higher costs for opening a gap and lower costs for extending an existing gap. For amino acid sequences match scores are usually encoded in matrices like _PAM_ or _BLOSUM_. Thus, a more meaningful alignment for our example can be obtained by using the _BLOSUM62_ matrix, together with a gap open penalty of 10 and a gap extension penalty of 0.5:

In [ ]:
from Bio import Align
from Bio import SeqIO
from Bio.Align import substitution_matrices
seq1 = SeqIO.read("alpha.faa", "fasta")
seq2 = SeqIO.read("beta.faa", "fasta")
aligner = Align.PairwiseAligner()
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

alignments = aligner.align(seq1.seq, seq2.seq)
print("Alignment Score:", alignments[0].score)
print(alignments[0])

### Multiple Sequence Alignment

#### ClustalW

ClustalW is a popular command line tool for multiple sequence alignment (there is also a graphical interface called ClustalX). Biopython’s `Bio.Align.Applications` module has a wrapper for this alignment tool (and several others).

Before trying to use ClustalW from within Python, you should first install CLustalW by downloading the spesific version for your OS from http://www.clustal.org/download/current/ and try running the ClustalW yourself by hand at the command line, to familiarize yourself the other options.

##### Install ClustalW in linux

In [ ]:
# Installing pre-requisites
%%bash
sudo apt-get install -y build-essential
sudo apt-get install -y gpp g++ c++ kcc fcc gpp

# Downloading ClustalW2
wget http://www.clustal.org/download/current/clustalw-2.1.tar.gz

#Installing ClustalW2
tar xvzf clustalw-2.1.tar.gz
cd clustalw-2.1/

./configure
make
make install

#Now, you can run clustalw2 in the terminal by typing: 
clustalw2

You’ll find the Biopython wrapper is very faithful to the actual command line API:

In [ ]:
from Bio.Align.Applications import ClustalwCommandline
help(ClustalwCommandline)

For the most basic usage, all you need is to have a FASTA input file, such as [opuntia.fasta](https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/opuntia.fasta). This is a small FASTA file containing seven prickly-pear DNA sequences (from the cactus family Opuntia).

In [ ]:
from Bio.Align.Applications import ClustalwCommandline
cline = ClustalwCommandline("clustalw2", infile="opuntia.fasta")
print(cline)

Notice here we have given the executable name as clustalw2, indicating we have version two installed, which has a different filename to version one (clustalw, the default). Fortunately both versions support the same set of arguments at the command line (and indeed, should be functionally identical).

You may find that even though you have ClustalW installed, the above command doesn’t work – you may get a message about “command not found” (especially on Windows). This indicated that the ClustalW executable is not on your PATH (an environment variable, a list of directories to be searched). You can either update your PATH setting to include the location of your copy of ClustalW tools (how you do this will depend on your OS), or simply type in the full path of the tool.

When you run the command line tool like this via the Biopython wrapper, it will wait for it to finish, and check the return code. If this is non zero (indicating an error), an exception is raised. The wrapper then returns two strings, stdout and stderr.

In the case of ClustalW, when run at the command line all the important output is written directly to the output files. Everything normally printed to screen while you wait (via stdout or stderr) is boring and can be ignored (assuming it worked).

What we care about are the two output files, the alignment and the guide tree. We didn’t tell ClustalW what filenames to use, but it defaults to picking names based on the input file. In this case the output should be in the file opuntia.aln. You should be able to work out how to read in the alignment using `Bio.AlignIO` by now:

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("opuntia.aln", "clustal")
print(alignment)

You’ll notice in the above output the sequences have been truncated. We could instead write our own code to format this as we please by iterating over the rows as SeqRecord objects:

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("opuntia.aln", "clustal")
print("Alignment length %i" % alignment.get_alignment_length())
for record in alignment:
    print("%s - %s" % (record.id, record.seq))

In case you are interested, the `opuntia.dnd` file ClustalW creates is just a standard _Newick tree_ file, and `Bio.Phylo` can parse these:

In [ ]:
from Bio import Phylo
tree = Phylo.read("opuntia.dnd", "newick")
Phylo.draw_ascii(tree)

## Homework

Prepare a Jupyter notebook to achive the goals below and upload to 'Homeworks/Week_06' folder under Google Drive directory of the course.

* Download protein sequences of TP53 from at least 100 different organisms
* Perform multiple sequence alignment and draw a phylogentic tree
* Find the correlated regions of TP53 protein
* Find the domains of correlated regions